In [57]:
import pandas as pd

import tkinter as tk
from tkinter import filedialog as fd
import dataframe_image as dfi

In [58]:
# open file dialog
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()
file_name = fd.askopenfilename(parent=root, filetypes=[(".xlsx", ".xlsx")])
root.destroy()

In [59]:
file_name

'D:/OneDrive - Ingenio Azucarero Guabira S.A/UTEA_2023/PLANOS_GRAL/01_ESTIMATIVAS_PRODUCCION_INST_50/DATA_INTERSECT.xlsx'

In [61]:
data = pd.read_excel(file_name)

In [62]:
data.head(3)

,OBJECTID,FID_Raster,Id,gridcode,FID_LOTES_,id_1,unidad_01,unidad_02,unidad_03,unidad_04,...,fs,textura,financia,soca,cultivo,zona,area,Shape_Leng,Shape_Area,cod_prop
0,1,1,1,3,1830,000017280000C3.2,1728,CAMPO_VERDE--PEREIRA_VALDEMAR,10770,PEREIRA_VALDEMAR,...,2018-06-20,FY,NaN,4,canha,50,0.005462,0.000311,4.631536e-09,1728
1,2,2,2,4,1830,000017280000C3.2,1728,CAMPO_VERDE--PEREIRA_VALDEMAR,10770,PEREIRA_VALDEMAR,...,2018-06-20,FY,NaN,4,canha,50,0.009242,0.000347,7.837441e-09,1728
2,3,3,3,3,1830,000017280000C3.2,1728,CAMPO_VERDE--PEREIRA_VALDEMAR,10770,PEREIRA_VALDEMAR,...,2018-06-20,FY,NaN,4,canha,50,0.005567,0.000311,4.720707e-09,1728


In [65]:
lista_cod_props = data['unidad_01']
lista_cod_props = list(set(list(lista_cod_props)))
print(lista_cod_props)

[8, 13, 14, 1551, 1552, 17, 1043, 22, 23, 1049, 1561, 27, 29, 30, 1567, 1566, 551, 558, 47, 559, 50, 52, 53, 54, 55, 63, 1090, 579, 72, 590, 591, 1615, 1618, 1619, 84, 94, 1645, 114, 115, 629, 122, 1151, 132, 141, 142, 658, 669, 674, 1702, 1705, 1707, 183, 184, 1719, 700, 1213, 1214, 1727, 1728, 1729, 1219, 197, 1221, 198, 721, 722, 1748, 216, 217, 219, 1777, 754, 755, 256, 257, 259, 267, 781, 270, 271, 782, 284, 288, 300, 306, 307, 308, 1335, 1336, 1337, 314, 830, 324, 325, 326, 327, 328, 343, 345, 865, 877, 375, 376, 1421, 1430, 1442, 423, 1447, 1450, 1453, 1454, 1474, 969, 1491, 985, 986, 1499, 1498, 1520, 1013, 1015, 506]


In [78]:
categoria = ['Nulo','Muy Bajo','Bajo Medio','Bajo','Medio','Medio Alto','Alto','Muy Alto']
colores = ['','','','','','','','']

def styling_specific_cell(x,row_idx,col_idx):
    color = ['background-color: #ff2200;',
             'background-color: #ff6f00;',
             'background-color: #ffa600;',
             'background-color: #ffe500;',
             'background-color: #d6e600;',
             'background-color: #8bb500;',
             'background-color: #498a00;',
             'background-color: #006100;']
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] =  color
    return df_styler

def set_blod_categori(x,row_idx,col_idx):
    color = ['font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;']
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] =  color
    return df_styler

def set_blod_row(x,row_idx,col_idx):
    color = 'font-weight:bold; background-color: #9BC2E6; color: black;'
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] =  color
    return df_styler

headers = {
    'selector': 'tr:hover',
    'props': [('background-color', '#9BC2E6'), ('color','black'), ('border','solid 1px')]
}

In [ ]:
def complete_index(df):
    index_missing = set(range(1, 9)) - set(df.index)
    print(index_missing)
    for i in index_missing:
        df.loc[i] = [0]
    df = df.sort_index()
    return df

In [156]:
for i in lista_cod_props:
    cod = i
    df = data[data['unidad_01'] == cod]
    df_aux = df[['gridcode', 'area']]
    
    df_aux = df_aux.groupby(['gridcode']).sum('area')
    df_aux = df_aux.rename(columns={'area':'AREA (ha)'})
    df_aux = complete_index(df_aux)
    #print(df_aux)
    df_aux.insert(loc=0, column='CATEGORIA', value=categoria)
    
    df_aux.insert(loc=1, column='COLOR', value=colores)
    df_aux['%']=(df_aux['AREA (ha)']/df_aux['AREA (ha)'].sum())*100
    df_aux.loc['Total'] = df_aux.sum(axis=0)
    df_aux.at['Total','CATEGORIA'] = 'TOTAL'
    df_aux['TCH (tn/ha)'] = ' '
    df_aux.at['Total','COLOR'] = ''
    
    formato = {'AREA (ha)':'{:.2f}', '%':'{:.2f} %'}
    i = pd.IndexSlice[df_aux.loc[(df_aux['%']<99.999999)].index, '%']
    s = df_aux.style.format(formato)\
        .bar(subset=i, color='#10CB23')\
        .hide_index()\
        .apply(styling_specific_cell, row_idx = [0,1,2,3,4,5,6,7], col_idx = 1, axis = None)\
        .apply(set_blod_categori, row_idx = [0,1,2,3,4,5,6,7,8], col_idx = 0, axis = None)\
        .apply(set_blod_row, row_idx = 8, col_idx = [0,1,2,3], axis = None)\
        .set_table_styles([headers])\
        .set_table_styles([{'selector': 'th',
                           'props': [('background-color', '#f8f8f8'),
                                     ('color', '#333'),
                                     ('border', '1px solid #333')]}])
    s = s.set_properties(border="1px solid black")
    tabla = s
    dfi.export(tabla, 'imgs/' + str(cod) + '.png')

{1, 2}
set()
set()
{1, 2, 3}
{8}
set()
set()
{1}
set()
{1}
{8, 7}
{1}
set()
set()
{8, 1, 2}
{1, 2}
set()
set()
set()
set()
set()
{1, 2}
{1}
set()
set()
set()
{8, 6, 7}
set()
set()
set()
set()
set()
set()
set()
{1}
set()
{1, 2, 3, 5}
set()
{1}
set()
set()
{8, 7}
set()
{1}
{1}
set()
set()
set()
{1}
set()
{1}
set()
set()
{1}
set()
{1, 2, 3}
{1, 2}
set()
set()
set()
{8}
set()
set()
set()
{1, 2, 3}
set()
{1}
set()
set()
set()
{8, 7}
{1, 2}
set()
{1}
set()
set()
set()
{1}
set()
{1}
set()
set()
{1}
set()
set()
{1}
set()
{1, 2}
{8, 1, 2, 3}
set()
{1}
set()
set()
set()
set()
set()
{1}
set()
{1}
set()
{1}
set()
{1, 2}
{1}
{1}
{1}
{8, 1}
set()
set()
{1}
set()
{8}
set()
{8, 1}
{1}
set()
{1, 2}
{1}
{1}
{1}
set()
set()


In [146]:
d = complete_index(aux)
d

set()


,AREA (ha)
gridcode,
1,1.643796
2,0.901373
3,4.864175
4,16.771687
5,37.542967
6,33.898003
7,17.471084
8,9.652222


In [127]:
aux.index

Int64Index([1, 2, 3, 4, 5, 6, 7, 8], dtype='int64', name='gridcode')

In [128]:
cod = lista_cod_props[1]
cod

13

In [129]:
# selecciona una propiedad por el codigo
df = data[data['unidad_01'] == 13]
df

,OBJECTID,FID_Raster,Id,gridcode,FID_LOTES_,id_1,unidad_01,unidad_02,unidad_03,unidad_04,...,fs,textura,financia,soca,cultivo,zona,area,Shape_Leng,Shape_Area,cod_prop
105753,105754,100014,100014,6,736,0000130000L17.2,13,GUAYABOCHI--AGUILERA,86,AGUILERA_TARADELLES_JOSE_LUIS,...,2019-05-28,F,IAG,3,canha,50,0.008729,0.000413,7.424122e-09,13
105789,105790,100049,100049,4,736,0000130000L17.2,13,GUAYABOCHI--AGUILERA,86,AGUILERA_TARADELLES_JOSE_LUIS,...,2019-05-28,F,IAG,3,canha,50,0.009488,0.000359,8.069703e-09,13
105790,105791,100050,100050,5,736,0000130000L17.2,13,GUAYABOCHI--AGUILERA,86,AGUILERA_TARADELLES_JOSE_LUIS,...,2019-05-28,F,IAG,3,canha,50,0.009488,0.000359,8.069703e-09,13
105829,105830,100087,100087,5,736,0000130000L17.2,13,GUAYABOCHI--AGUILERA,86,AGUILERA_TARADELLES_JOSE_LUIS,...,2019-05-28,F,IAG,3,canha,50,0.014763,0.000498,1.255657e-08,13
105874,105875,100127,100127,4,736,0000130000L17.2,13,GUAYABOCHI--AGUILERA,86,AGUILERA_TARADELLES_JOSE_LUIS,...,2019-05-28,F,IAG,3,canha,50,0.013235,0.000488,1.125739e-08,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149958,149959,101419,101419,7,1938,0000130000L17.1,13,GUAYABOCHI--AGUILERA,86,AGUILERA_TARADELLES_JOSE_LUIS,...,2022-04-18,F,IAG,0,canha,50,0.004532,0.001672,3.854332e-09,13
149959,149960,103034,103034,4,129,0000130000L12.2,13,GUAYABOCHI--AGUILERA,14558,VACA_DIEZ_ATALA_MARIA_ALEJANDRA,...,2020-05-10,FA,IAG,2,canha,50,0.000005,0.000684,3.943020e-12,13
149960,149961,103034,103034,4,1444,0000130000L13,13,GUAYABOCHI--AGUILERA,86,AGUILERA_TARADELLES_JOSE_LUIS,...,2021-05-01,FA,IAG,1,canha,50,0.000005,0.000684,3.943020e-12,13
149961,149962,104957,104957,5,129,0000130000L12.2,13,GUAYABOCHI--AGUILERA,14558,VACA_DIEZ_ATALA_MARIA_ALEJANDRA,...,2020-05-10,FA,IAG,2,canha,50,0.000002,0.000288,2.024552e-12,13


In [130]:
aux = df[['gridcode', 'area']]
aux

,gridcode,area
105753,6,0.008729
105789,4,0.009488
105790,5,0.009488
105829,5,0.014763
105874,4,0.013235
...,...,...
149958,7,0.004532
149959,4,0.000005
149960,4,0.000005
149961,5,0.000002


In [131]:
aux = aux.groupby(['gridcode']).sum('area')
aux = aux.rename(columns={'area':'AREA (ha)'})
aux

,AREA (ha)
gridcode,
1,1.643796
2,0.901373
3,4.864175
4,16.771687
5,37.542967
6,33.898003
7,17.471084
8,9.652222


In [86]:
categoria = ['Nulo','Muy Bajo','Bajo Medio','Bajo','Medio','Medio Alto','Alto','Muy Alto']
colores = ['','','','','','','','']

In [87]:
aux.insert(loc=0, column='CATEGORIA', value=categoria)
aux.insert(loc=1, column='COLOR', value=colores)
aux['%']=(aux['AREA (ha)']/aux['AREA (ha)'].sum())*100
aux.loc['Total'] = aux.sum(axis=0)
aux.at['Total','CATEGORIA'] = 'TOTAL'
aux['TCH (tn/ha)'] = ' '

In [88]:
aux

,CATEGORIA,COLOR,AREA (ha),%,TCH (tn/ha)
gridcode,,,,,
1,Nulo,,1.643796,1.339192,
2,Muy Bajo,,0.901373,0.734344,
3,Bajo Medio,,4.864175,3.962819,
4,Bajo,,16.771687,13.663811,
5,Medio,,37.542967,30.586071,
6,Medio Alto,,33.898003,27.616537,
7,Alto,,17.471084,14.233606,
8,Muy Alto,,9.652222,7.863618,
Total,TOTAL,,122.745305,100.000000,


In [89]:
aux.at['Total','COLOR'] = ''

In [90]:
def styling_specific_cell(x,row_idx,col_idx):
    color = ['background-color: #ff2200;',
             'background-color: #ff6f00;',
             'background-color: #ffa600;',
             'background-color: #ffe500;',
             'background-color: #d6e600;',
             'background-color: #8bb500;',
             'background-color: #498a00;',
             'background-color: #006100;']
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] =  color
    return df_styler

def set_blod_categori(x,row_idx,col_idx):
    color = ['font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;']
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] =  color
    return df_styler

def set_blod_row(x,row_idx,col_idx):
    color = 'font-weight:bold; background-color: #9BC2E6; color: black;'
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] =  color
    return df_styler

headers = {
    'selector': 'tr:hover',
    'props': [('background-color', '#9BC2E6'), ('color','black'), ('border','solid 1px')]
}

In [91]:
aux

,CATEGORIA,COLOR,AREA (ha),%,TCH (tn/ha)
gridcode,,,,,
1,Nulo,,1.643796,1.339192,
2,Muy Bajo,,0.901373,0.734344,
3,Bajo Medio,,4.864175,3.962819,
4,Bajo,,16.771687,13.663811,
5,Medio,,37.542967,30.586071,
6,Medio Alto,,33.898003,27.616537,
7,Alto,,17.471084,14.233606,
8,Muy Alto,,9.652222,7.863618,
Total,TOTAL,,122.745305,100.000000,


In [92]:
formato = {'AREA (ha)':'{:.2f}', '%':'{:.2f} %'}
i = pd.IndexSlice[aux.loc[(aux['%']<99.999999)].index, '%']

s = aux.style.format(formato)\
    .bar(subset=i, color='#10CB23')\
    .hide_index()\
    .apply(styling_specific_cell, row_idx = [0,1,2,3,4,5,6,7], col_idx = 1, axis = None)\
    .apply(set_blod_categori, row_idx = [0,1,2,3,4,5,6,7,8], col_idx = 0, axis = None)\
    .apply(set_blod_row, row_idx = 8, col_idx = [0,1,2,3], axis = None)\
    .set_table_styles([headers])\
    .set_table_styles([{'selector': 'th',
                           'props': [('background-color', '#f8f8f8'),
                                     ('color', '#333'),
                                     ('border', '1px solid #333')]}])
s = s.set_properties(border="1px solid black")
tabla = s
tabla

CATEGORIA,COLOR,AREA (ha),%,TCH (tn/ha)
Nulo,,1.64,1.34 %,
Muy Bajo,,0.90,0.73 %,
Bajo Medio,,4.86,3.96 %,
Bajo,,16.77,13.66 %,
Medio,,37.54,30.59 %,
Medio Alto,,33.90,27.62 %,
Alto,,17.47,14.23 %,
Muy Alto,,9.65,7.86 %,
TOTAL,,122.75,100.00 %,


In [55]:
dfi.export(tabla, 'tabla.png')